In [63]:
# simple code for parsed output

In [64]:
# Import of the packages

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
from pydantic import BaseModel, Field

In [2]:
#aded prompts for templates
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate

In [3]:
# add parsers

In [4]:
from langchain.output_parsers import PydanticOutputParser

In [5]:
# load env and vriables
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [6]:
# main function

In [7]:
def ask_chat(model_name, template, variables, parser):
    llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=model_name, temperature=0.7)
    message = HumanMessagePromptTemplate.from_template(template=template)
    prompt = ChatPromptTemplate.from_messages(messages=[message])
    # Formatting the prompt with the given variables
    prompt_with_values = prompt.format_prompt(**variables) 
    # creation of messge
    messages = prompt_with_values.to_messages()
    answer = llm(messages)
    output = parser.parse(answer.content)
    
    return output

In [8]:
# create template

In [9]:
PROMPT_TEMPLATE = """
    Provide information availible about {person}.
    {output_format}
    """

In [10]:
#create output report

In [11]:
class OuputReport(BaseModel):
    person: str = Field(description="name of the person")
    nationality: str = Field(description="nationality of the person if it is known")
    occupation: str = Field(description="occupation of the person if it is known")
    informations: str = Field(description="all other information that is known")

    def __str__(self):
        msgs = [f"{k}: \t {v}" for k,v in self.__dict__.items()]
        return '\n'.join(msgs)
      

In [12]:
# creation of the parser

In [13]:
parser = PydanticOutputParser(pydantic_object=OuputReport)

In [14]:
variables = {"person": "Dimitry Sytyi",
            "output_format": parser.get_format_instructions()}

In [15]:
answer =  ask_chat(model_name="gpt-4o",  template=PROMPT_TEMPLATE, variables=variables, parser=parser)

C:\Users\micha\PycharmProjects\AIPlayground\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [16]:
print(answer)

person: 	 Dimitry Sytyi
nationality: 	 Russian
occupation: 	 Businessman
informations: 	 Dimitry Sytyi is known to be associated with the Wagner Group, a Russian private military company. He manages operations in the Central African Republic. Sytyi was reportedly injured in a bomb attack in December 2022.
